# Installation (20~40m)

In [1]:
# Install Python libraries
print("Installing Python libraries...\n")
!pip install plyfile Pillow==9.5.0 pycolmap mediapy

# Install system packages required for COLMAP
print("\nInstalling system packages for COLMAP...\n")
!apt-get install colmap ffmpeg libglm-dev

# Navigate back to the root directory
%cd /content

# Clone HLOC and install its dependencies
print("\nCloning HLOC and installing dependencies...\n")
!git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd Hierarchical-Localization
!pip install --quiet -e .
!pip install --quiet --upgrade plotly

# Navigate back to the root directory
%cd /content

# Clone gaussian-splatting
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

print("\nInstallation completed!")

Installing Python libraries...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0



Installing system packages for COLMAP...

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxsparse3 libevdev2
  libfreeimage3 libgflags2.2 libgoogle-glog0v5 libgudev-1.0-0 libinput-bin libinput10 libjxr0
  libmd4c0 libmetis5 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5
  libqt5widgets5 libraw20 libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common libwacom9
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0
  libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0 qt5-gtk-platformtheme qttranslations5-l10n
Suggested packages:
  qt5-image-formats-plugins qtwayland5
The following NEW packages will be installed:
  colmap libamd2 libcamd2 libccolamd2 libceres2 libcholmod3 libcolamd2 libcxspar

In [5]:
import glob
import os
import io
import sys
import shutil
import glob
import cv2
import sqlite3
import numpy as np
import torch
import matplotlib.pyplot as plt
import mediapy as media
from pathlib import Path
from time import time
from PIL import Image, ImageFilter, ImageChops
import pycolmap

%cd /content/Hierarchical-Localization
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval, colmap_from_nvm, pairs_from_exhaustive, pairs_from_covisibility
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

import sys
sys.path.append('/notebooks/Hierarchical-Localization/third_party')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/d2net')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/deep-image-retrieval')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/r2d2')
sys.path.append('/notebooks/Hierarchical-Localization/third_party/SuperGluePretrainedNetwork')
%cd /content

/content/Hierarchical-Localization
/content


In [6]:
%cd /content
connection = sqlite3.connect('/content/dbs/db_session_0AC6D7F9-C85D-418C-A21C-5934530D9782.sqlite')

# Get all data
cursor = connection.cursor()
cursor.execute("SELECT image_blob FROM maptodon_raw")
results = cursor.fetchall()

# Delete working image, and create a new one
!rm -rf working
!mkdir working
!mkdir working/tmp_input

# Process the search results
image_hashes = {}
if len(results) > 0:
	for i, row in enumerate(results):
		image_data = row[0]

		# Save to working directory
		image = Image.open(io.BytesIO(image_data))

		image.save("./working/tmp_input/image_{:05d}.png".format(i))
	print("Saved " + str(len(results)) + " images to working directory.")
else:
	print("No results found.")

# Close the database connection
connection.close()

/content


DatabaseError: ignored

In [ ]:
!rm -rf ./working/input
!mkdir ./working/input

images = glob.glob("./working/tmp_input/*.png")

# Maximum number of images to copy
M = 20
step = len(images) // M

for i, image in enumerate(images):
	if i % step == 0:
		shutil.copy(image, f"./working/input/{i:05d}.png")
images = glob.glob("./working/input/*.png")
print(f"There are {len(images)} images in the working directory.")


In [ ]:
%%time
%cd /content

location = Path('./working')
images = location / 'input'
image_list = [path.split(str(images.stem) + "/")[-1] for path in sorted(glob.glob(str(images / '*')))]
outputs = location / 'sfm'
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_max']
matcher_conf = match_features.confs['superglue']

extract_features.main(feature_conf, images, image_list=image_list, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=image_list)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=image_list)

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

In [ ]:
%%time
print("\nRunning COLMAP image undistorter...\n")

# Running COLMAP image undistorter
undistorted_path = location / "undistorted"
!rm -rf {undistorted_path}

!colmap image_undistorter \
	--image_path /notebooks/working/input \
	--input_path {sfm_dir} \
	--output_path {undistorted_path} \
	--output_type COLMAP \
	# --max_image_size 2000

!mv {undistorted_path}/sparse {undistorted_path}/sparse_tmp
!mkdir {undistorted_path}/sparse
!mv {undistorted_path}/sparse_tmp {undistorted_path}/sparse/0

In [ ]:
!python /content/gaussian-splatting/train.py -s {undistorted_path} -r 2